# 第10章: 機械翻訳
本章では，日本語と英語の翻訳コーパスである[京都フリー翻訳タスク (KFTT)](http://www.phontron.com/kftt/index-ja.html)を用い，ニューラル機械翻訳モデルを構築する．ニューラル機械翻訳モデルの構築には，[fairseq](https://github.com/pytorch/fairseq)，[Hugging Face Transformers](https://github.com/huggingface/transformers)，[OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py)などの既存のツールを活用せよ．

GPUの使用制限がかかるので，データの永続化をする  
（ドライブに作業フォルダを保存する）

In [1]:
# 作業用フォルダの作成
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

Mounted at /content/drive
/content/drive/My Drive/work


## 90. データの準備
機械翻訳のデータセットをダウンロードせよ．訓練データ，開発データ，評価データを整形し，必要に応じてトークン化などの前処理を行うこと．ただし，この段階ではトークンの単位として形態素（日本語）および単語（英語）を採用せよ．

In [34]:
## データセットダウンロード
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz

--2021-07-21 04:06:54--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz’

kftt-data-1.0.tar.g 100%[===================>]  94.65M  29.9MB/s    in 3.2s    

2021-07-21 04:06:58 (29.9 MB/s) - ‘kftt-data-1.0.tar.gz’ saved [99246893/99246893]



In [35]:
!gunzip kftt-data-1.0.tar.gz

gzip: kftt-data-1.0.tar already exists; do you wish to overwrite (y or n)? y


In [36]:
!tar -xvf kftt-data-1.0.tar

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/tok/kyoto-dev.ja
kftt-data-1.0/data/tok/kyoto-train.cln.en
kftt-data-1.0/data/tok/kyoto-dev.en
kftt-data-1.0/data/tok/kyoto-train.en
kftt-data-1.0/data/tok/kyoto-tune.ja
kftt-data-1.0/data/tok/kyoto-train.cln.ja
kftt-data-1.0/data/tok/kyoto-train.ja
kftt-data-1.0/data/tok/kyoto-test.ja
kftt-data-1.0/data/tok/kyoto-test.en
kftt-data-1.0/README.txt


In [ ]:
!head -10 ./kftt-data-1.0/data/tok/kyoto-train.en

Known as Sesshu ( 1420 - 1506 ) , he was an ink painter and Zen monk active in the Muromachi period in the latter half of the 15th century , and was called a master painter .
He revolutionized the Japanese ink painting .
He was given the posthumous name " Toyo " or " Sesshu ( 拙宗 ) . "
Born in Bicchu Province , he moved to Suo Province after entering SShokoku-ji Temple in Kyoto .
Later he accompanied a mission to Ming Dynasty China and learned Chinese ink painting .
His works were many , including not only Chinese-style landscape paintings , but also portraits and pictures of flowers and birds .
His bold compositions and strong brush strokes constituted an extremely distinctive style .
6 of his extant works are designated national treasures . Indeed , he is considered to be extraordinary among Japanese painters .
For this reason , there are a great many artworks that are attributed to him , such as folding screens with pictures of flowers and that birds are painted on them .
There are m

In [ ]:
!head -10 ./kftt-data-1.0/data/tok/kyoto-train.ja

雪舟 （ せっしゅう 、 1420 年 （ 応永 27 年 ） - 1506 年 （ 永正 3 年 ） ） は 号 で 、 15 世紀 後半 室町 時代 に 活躍 し た 水墨 画 家 ・ 禅僧 で 、 画聖 と も 称え られ る 。
日本 の 水墨 画 を 一変 さ せ た 。
諱 は 「 等楊 （ とうよう ） 」 、 もしくは 「 拙宗 （ せっしゅう ） 」 と 号 し た 。
備中 国 に 生まれ 、 京都 ・ 相国 寺 に 入 っ て から 周防 国 に 移 る 。
その 後 遣明 使 に 随行 し て 中国 （ 明 ） に 渡 っ て 中国 の 水墨 画 を 学 ん だ 。
作品 は 数 多 く 、 中国 風 の 山水 画 だけ で な く 人物 画 や 花鳥 画 も よ く し た 。
大胆 な 構図 と 力強 い 筆線 は 非常 に 個性 的 な 画風 を 作り出 し て い る 。
現存 する 作品 の うち 6 点 が 国宝 に 指定 さ れ て お り 、 日本 の 画家 の なか で も 別格 の 評価 を 受け て い る と いえ る 。
この ため 、 花鳥 図 屏風 など に 「 伝 雪舟 筆 」 さ れ る 作品 は 大変 多 い 。
真筆 で あ る か 専門 家 の 間 で も 意見 の 分かれ る もの も 多々 あ る 。


### 前処理

### どうやら，[fairseq](https://github.com/pytorch/fairseq)が便利らしい．
コマンドを打つだけで，学習や予測ができるので

### fairseqとは
Facebookの人工知能研究チームが開発している，機械翻訳用のフレームワークです．Facebookが開発元ということもあり，Pytorchがベースになっています．  
最近はHuggingfaceのTransformersが人気でTransformerモデルを扱うならPytorchを使用していたため，こちらをフレームワークとして選びました．  
その他の機械翻訳フレームワークとしては，MarianNMT，OpenNMT（こちらもPytorchベース）などがあります．基本的な機能はどのフレームワークも大差ない印象ですが，論文実装のコードはfairseqが選ばれている場合が多いらしいです．

In [8]:
## fairseqのインストール
!pip install fairseq

     |████████████████████████████████| 1.7 MB 8.3 MB/s 
     |████████████████████████████████| 54 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 40.1 MB/s 
     |████████████████████████████████| 112 kB 42.0 MB/s 
     |████████████████████████████████| 74 kB 3.8 MB/s 
     |████████████████████████████████| 636 kB 50.1 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=1070a9162a93e6182a970fcb7f954cedd1969d8a77cbfb78cea854844fff2680
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os

fairseq-preprocessはテキストデータをバイナリ化して保存します。今回はすでにspaceでtokenizeされているのでspaceを使いました。

In [ ]:
!ls

data91	kftt-data-1.0  kftt-data-1.0.tar


In [ ]:
!fairseq-preprocess -s ja -t en \
    --trainpref  ./kftt-data-1.0/data/tok/kyoto-train \
    --validpref  ./kftt-data-1.0/data/tok/kyoto-dev \
    --testpref ./kftt-data-1.0/data/tok/kyoto-test \
    --destdir data91  \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

2021-07-20 02:30:11 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data91', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='ja', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='./kftt-data-1.0/data/tok/kyoto-test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=5, thresholdtgt=5, tokenizer=None, tp

In [ ]:
!ls kftt-data-1.0/data/tok/

kyoto-dev.en  kyoto-test.en  kyoto-train.cln.en  kyoto-train.en  kyoto-tune.en
kyoto-dev.ja  kyoto-test.ja  kyoto-train.cln.ja  kyoto-train.ja  kyoto-tune.ja


## 91. 機械翻訳モデルの訓練
90で準備したデータを用いて，ニューラル機械翻訳のモデルを学習せよ（ニューラルネットワークのモデルはTransformerやLSTMなど適当に選んでよい）．

学習の設定は以下に箇条書きした．  
（ハイパーパラメータは後にチューニングする）

*   モデル：LSTM
*   optimizer : adam
*   学習率：1e-5
*   ドロップアウト：0.3
*   最大エポック数：5　or 10
*   損失：交差エントロピー











In [22]:
!pip install sacremoses

In [ ]:
!fairseq-train data91 \
    --save-dir checkpoints/kftt.ja-en/ \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-epoch 5 \
    --eval-bleu \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

## --eval-bleu-detok moses \ トークン化解除
##　--eval-bleu-remove-bpe \　BLEUを計算する前にBPEを削除する
##　--eval-bleu-print-samples \　検証中にサンプルした世代を出力する
##　--best-checkpoint-metric bleu \　最良のチェックポイントを保存するために使用するメトリック=bleuに指定
##　--maximize-best-checkpoint-metric 

2021-07-20 14:09:38 | INFO | fairseq_cli.train | Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='lstm', batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data91', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention='1', decoder_dropout_in=0.3, decoder_dropout_out=0.3, decoder_embed_dim=512, decoder_embed_path=None, decoder_freeze_embed=False, decoder_hidden_size=512, decoder_layers=1, decoder_out_embed_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0

epoch=5のとき，BLEUはだいたい12くらいになった．

引数の説明

*   data91: 第一引数はモデルの保存先を指定します。これがデフォルトのようです

*   --lr: learning rateはデータ、バッチサイズなどで大きく適切な値が変わります。1e-5がよく使われているらしいです．

*   --warmup-updates: このパラメータの回数分のminibatchを処理した段階でlrが最大値になります。
1epochのminibatchの数は後述のパラメータで大きく変わるので、変更し忘れないように注意







In [ ]:
## epoch=10で再度やってみる　→　GPU使用制限のため断念
#!fairseq-train data91 \
#    --save-dir checkpoints/kftt.ja-en/epoch10/ \
#    --arch lstm --share-decoder-input-output-embed \
#    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
#    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
#    --dropout 0.3 --weight-decay 0.0001 \
#    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
#    --max-tokens 4096 \
#    --max-epoch 10 \
#    --eval-bleu \
#    --eval-bleu-detok moses \
#    --eval-bleu-remove-bpe \
#    --eval-bleu-print-samples \
#    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

## 92. 機械翻訳モデルの適用
91で学習したニューラル機械翻訳モデルを用い，与えられた（任意の）日本語の文を英語に翻訳するプログラムを実装せよ．



fairseq-interactiveを使って，翻訳する．  

https://fairseq.readthedocs.io/en/latest/command_line_tools.html#fairseq-interactive

In [ ]:
!fairseq-interactive data91/ \
  --path checkpoints/kftt.ja-en/checkpoint_best.pt \
  < ./kftt-data-1.0/data/tok/kyoto-test.ja \
   | grep '^H' | cut -f3 > 92.out

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [5]:
!head 92.out -n 10

<unk>
Dogen was a Zen monk in the early Kamakura period .
He was the founder of the Soto sect .
His name was also used in his later years .
He was given the title of <unk> ( an honorific title for the Emperor ) .
His posthumous Buddhist name was <unk> Kokushi ( <unk> ) and <unk> Daishi ( a priest ) .
In general , he was called Dogen Dogen .
It is said that he spread the custom of eating food and food in Japan .
There is also a theory that he brought back to <unk> <unk> ( <unk> ) .
There are many theories about the birth of Dogen , but there are various theories on the fact that he was born as a direct descendant of Michichika TSUCHIMIKADO ( MINAMOTO no Michichika and Michichika KOGA ) .


## 93. BLEUスコアの計測
91で学習したニューラル機械翻訳モデルの品質を調べるため，評価データにおけるBLEUスコアを測定せよ．



In [ ]:
!fairseq-score --sys 92.out --ref kftt-data-1.0/data/tok/kyoto-test.en

Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-test.en', sacrebleu=False, sentence_bleu=False, sys='92.out')
BLEU4 = 14.72, 42.9/18.6/10.1/5.8 (BP=1.000, ratio=1.024, syslen=27384, reflen=26734)


## 94. ビーム探索
91で学習したニューラル機械翻訳モデルで翻訳文をデコードする際に，ビーム探索を導入せよ．ビーム幅を1から100くらいまで適当に変化させながら，開発セット上のBLEUスコアの変化をプロットせよ．



In [ ]:
!mkdir 94

In [ ]:
#beam size = 1
!fairseq-interactive \
  --path checkpoints/kftt.ja-en/checkpoint_best.pt \
  --beam 1 data91 \
  < kftt-data-1.0/data/tok/kyoto-dev.ja | grep '^H' | cut -f3 > 94/beam1.out
!fairseq-score --sys 94/beam1.out --ref kftt-data-1.0/data/tok/kyoto-dev.en

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-dev.en', sacrebleu=False, sentence_bleu=False, sys='94/beam1.out')
BLEU4 = 10.33, 38.4/14.4/6.5/3.2 (BP=1.000, ratio=1.134, syslen=27577, reflen=24309)


In [ ]:
#beam size = 5
!fairseq-interactive \
  --path checkpoints/kftt.ja-en/checkpoint_best.pt \
  --beam 5 data91 \
  < kftt-data-1.0/data/tok/kyoto-dev.ja | grep '^H' | cut -f3 > 94/beam5.out
!fairseq-score --sys 94/beam5.out --ref kftt-data-1.0/data/tok/kyoto-dev.en

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-dev.en', sacrebleu=False, sentence_bleu=False, sys='94/beam5.out')
BLEU4 = 12.06, 41.5/16.4/7.8/4.0 (BP=1.000, ratio=1.056, syslen=25681, reflen=24309)


In [ ]:
#beam size = 10
!fairseq-interactive \
  --path checkpoints/kftt.ja-en/checkpoint_best.pt \
  --beam 10 data91 \
  < kftt-data-1.0/data/tok/kyoto-dev.ja | grep '^H' | cut -f3 > 94/beam10.out
!fairseq-score --sys 94/beam10.out --ref kftt-data-1.0/data/tok/kyoto-dev.en

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-dev.en', sacrebleu=False, sentence_bleu=False, sys='94/beam10.out')
BLEU4 = 11.97, 41.5/16.3/7.7/4.0 (BP=1.000, ratio=1.038, syslen=25228, reflen=24309)


## 95. サブワード化
トークンの単位を単語や形態素からサブワードに変更し，91-94の実験を再度実施せよ．



In [6]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.2 MB/s 


In [ ]:
import sentencepiece as spm
import re

spm.SentencePieceTrainer.Train('--input=kftt-data-1.0/data/orig/kyoto-train.ja --model_prefix=kyoto_ja --vocab_size=16000 --character_coverage=1.0')

sp = spm.SentencePieceProcessor()
sp.Load('kyoto_ja.model')

for src, dst in [
    ('kftt-data-1.0/data/orig/kyoto-train.ja', 'train.sub.ja'),
    ('kftt-data-1.0/data/orig/kyoto-dev.ja', 'dev.sub.ja'),
    ('kftt-data-1.0/data/orig/kyoto-test.ja', 'test.sub.ja'),
]:
    with open(src) as f, open(dst, 'w') as g:
        for x in f:
            x = x.strip()
            x = re.sub(r'\s+', ' ', x)
            x = sp.encode_as_pieces(x)
            x = ' '.join(x)
            print(x, file=g)

In [ ]:
!head train.sub.ja

▁ 雪 舟 ( せ っ しゅう 、 14 20 年 ( 応永 27 年 )- 150 6 年 ( 永正 3 年 ) ) は 号 で 、 15 世紀後半 室町時代に 活躍した 水墨画 家 ・ 禅僧 で 、 画 聖 とも 称え られる 。
▁日本の 水墨画 を 一 変 させた 。
▁諱は 「 等 楊 ( とう よう ) 」 、 もしくは 「 拙 宗 ( せ っ しゅう ) 」 と号した 。
▁ 備中国 に 生まれ 、 京都 ・ 相国寺 に入って から 周防国 に移る 。
▁その後 遣 明 使 に 随行 して 中国 ( 明 ) に渡って 中国の 水墨画 を学んだ 。
▁ 作品 は 数多く 、 中国 風の 山 水 画 だけでなく 人物 画 や 花鳥 画 も よく した 。
▁大 胆 な 構図 と 力 強い 筆 線 は非常に 個 性 的な 画 風 を作り 出している 。
▁ 現存する 作品 のうち 6 点 が 国宝 に指定され ており 、 日本の 画家 のなかで も 別 格 の 評価 を受けている といえる 。
▁このため 、 花鳥 図屏風 などに 「 伝 雪 舟 筆 」 される 作品 は 大変 多い 。
▁ 真 筆 である か 専門 家 の間で も 意見 の 分かれ るもの も 多 々 ある 。


In [ ]:
!pip install subword-nmt

In [ ]:
!subword-nmt learn-bpe -s 16000 < kftt-data-1.0/data/orig/kyoto-train.en > kyoto_en.codes
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-train.en > train.sub.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-dev.en > dev.sub.en
!subword-nmt apply-bpe -c kyoto_en.codes < kftt-data-1.0/data/orig/kyoto-test.en > test.sub.en

In [ ]:
!head train.sub.en

K@@ n@@ own as Ses@@ shu (14@@ 20 - 150@@ 6@@ ), he was an ink painter and Zen monk active in the Muromachi period in the latter half of the 15th century, and was called a master pain@@ ter.
He revol@@ ut@@ ion@@ ized the Japanese ink paint@@ ing.
He was given the posthumous name "@@ Toyo@@ " or "S@@ es@@ shu (@@ 拙@@ 宗@@ )."
Born in Bicchu Province, he moved to Suo Province after entering S@@ Shokoku-ji Temple in Kyoto.
Later he accompanied a mission to Ming Dynasty China and learned Chinese ink paint@@ ing.
His works were man@@ y, including not only Chinese-style landscape paintings, but also portraits and pictures of flowers and bird@@ s.
His b@@ old compos@@ itions and strong brush st@@ rok@@ es const@@ ituted an extremely distinctive style.
6 of his ext@@ ant works are designated national treasu@@ res. In@@ de@@ ed, he is considered to be extraordinary among Japanese pain@@ ters.
For this reason, there are a great many art@@ works that are attributed to him, such as folding scre@@ 

In [12]:
#前処理
!mkdir data91/kftt-bpe.ja-en
!mkdir kftt-data-1.0/data/bpe

mkdir: cannot create directory ‘data91/kftt-bpe.ja-en’: File exists
mkdir: cannot create directory ‘kftt-data-1.0/data/bpe’: File exists
mkdir: cannot create directory ‘checkpoints/kftt-bpe.ja-en’: File exists


In [10]:
!fairseq-preprocess --source-lang ja --target-lang en \
    --trainpref train.sub \
    --validpref dev.sub \
    --testpref test.sub \
    --destdir data91/kftt-bpe.ja-en/ \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --bpe subword_nmt \
    --workers 20

2021-07-21 01:47:07 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe='subword_nmt', checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data91/kftt-bpe.ja-en/', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='ja', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='test.sub', tgtdict=None, threshold_loss_scale=None, thresholdsrc=5, thresholdtgt=5, tokenizer=None, tpu=

**なぜか，学習時に以下のエラーが出るようになる(以前は同じ書き方ではエラーは出なかった．)**

`TypeError: forward() missing 1 required positional argument: 'prev_output_tokens'`

In [25]:
#学習
!mkdir checkpoints/kftt-bpe.ja-en
!fairseq-train data91/kftt-bpe.ja-en \
    --save-dir checkpoints/kftt-bpe.ja-en/ \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-epoch 5 \
    --eval-bleu \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

mkdir: cannot create directory ‘checkpoints/kftt-bpe.ja-en’: File exists
2021-07-21 02:09:25 | INFO | fairseq_cli.train | Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='lstm', batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data91/kftt-bpe.ja-en', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention='1', decoder_dropout_in=0.3, decoder_dropout_out=0.3, decoder_embed_dim=512, decoder_embed_path=None, decoder_freeze_embed=False, decoder_hidden_size=512, decoder_layers=1, decoder_out_embed_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_por

In [16]:
#翻訳
!fairseq-interactive data91/kftt-bpe.ja-en \
    --path checkpoints/kftt-bpe.ja-en/checkpoint_best.pt \
    --remove-bpe --bpe subword_nmt --bpe-codes $BPE_CODE\
    < ../kftt-data-1.0/data/tok/kyoto-test.ja \
    | grep '^H' | cut -f3 > 95.out

/bin/bash: ../kftt-data-1.0/data/tok/kyoto-test.ja: No such file or directory


In [17]:
#評価
!fairseq-score --sys 95.out --ref ../kftt-data-1.0/data/tok/kyoto-test.en

Namespace(ignore_case=False, order=4, ref='../kftt-data-1.0/data/tok/kyoto-test.en', sacrebleu=False, sentence_bleu=False, sys='95.out')
Traceback (most recent call last):
  File "/usr/local/bin/fairseq-score", line 8, in <module>
    sys.exit(cli_main())
  File "/usr/local/lib/python3.7/dist-packages/fairseq_cli/score.py", line 45, in cli_main
    assert os.path.exists(args.ref), "Reference file {} does not exist".format(args.ref)
AssertionError: Reference file ../kftt-data-1.0/data/tok/kyoto-test.en does not exist


## 96. 学習過程の可視化
Tensorboardなどのツールを用い，ニューラル機械翻訳モデルが学習されていく過程を可視化せよ．可視化する項目としては，学習データにおける損失関数の値とBLEUスコア，開発データにおける損失関数の値とBLEUスコアなどを採用せよ．



In [27]:
!mkdir checkpoints/kftt-bpe-tensorboard.ja-en

mkdir: cannot create directory ‘checkpoints/kftt-bpe-tensorboard.ja-en’: File exists


**なぜか，学習時に以下のエラーが出るようになる(以前は同じ書き方ではエラーは出なかった．)**

`TypeError: forward() missing 1 required positional argument: 'prev_output_tokens'`

In [28]:
!fairseq-train data91/kftt-bpe.ja-en \
    --save-dir checkpoints/kftt-bpe-tensorboard.ja-en/ \
    --tensorboard-logdir checkpoints/kftt-bpe-tensorboard.ja-en/ \
    --arch lstm --share-decoder-input-output-embed \
    --bpe subword_nmt \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-epoch 10 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

2021-07-21 02:10:28 | INFO | fairseq_cli.train | Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='lstm', batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe='subword_nmt', bpe_codes=None, bpe_separator='@@', broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data91/kftt-bpe.ja-en', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention='1', decoder_dropout_in=0.3, decoder_dropout_out=0.3, decoder_embed_dim=512, decoder_embed_path=None, decoder_freeze_embed=False, decoder_hidden_size=512, decoder_layers=1, decoder_out_embed_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, di

In [ ]:
!tensorboard --logdir ./checkpoints/kftt-bpe-tensorboard.ja-en/

2021-07-20 16:00:24.882580: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.5.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


## 97. ハイパー・パラメータの調整
ニューラルネットワークのモデルや，そのハイパーパラメータを変更しつつ，開発データにおけるBLEUスコアが最大となるモデルとハイパーパラメータを求めよ．

In [19]:
!mkdir checkpoints/kftt-search.ja-en/

mkdir: cannot create directory ‘checkpoints/kftt-search.ja-en/’: File exists


In [29]:
#学習
!mkdir checkpoints/kftt-search.ja-en/lr_5e-3/
!fairseq-train data91 \
    --save-dir checkpoints/kftt-search.ja-en/lr_5e-3 \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-epoch 5 \
    --eval-bleu \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

mkdir: cannot create directory ‘checkpoints/kftt-search.ja-en/lr_5e-3/’: File exists
2021-07-21 02:11:23 | INFO | fairseq_cli.train | Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='lstm', batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data91', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention='1', decoder_dropout_in=0.3, decoder_dropout_out=0.3, decoder_embed_dim=512, decoder_embed_path=None, decoder_freeze_embed=False, decoder_hidden_size=512, decoder_layers=1, decoder_out_embed_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-

In [32]:
#翻訳
!mkdir 97
!fairseq-interactive data91 \
    --path checkpoints/kftt-search.ja-en/lr_5e-3/checkpoint_best.pt \
    < kftt-data-1.0/data/tok/kyoto-test.ja \
    | grep '^H' | cut -f3 > 97/5e-3.out

mkdir: cannot create directory ‘97’: File exists
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [37]:
#評価 
!fairseq-score --sys 97/5e-3.out --ref kftt-data-1.0/data/tok/kyoto-test.en


Namespace(ignore_case=False, order=4, ref='kftt-data-1.0/data/tok/kyoto-test.en', sacrebleu=False, sentence_bleu=False, sys='97/5e-3.out')
BLEU4 = 15.54, 46.5/20.9/11.3/6.6 (BP=0.948, ratio=0.950, syslen=25390, reflen=26734)
